# Paying multiple scholarships from multiple Marlowe contracts


**Executive Summary**

This example demonstrates a multisig address simultaneously creating 100 contracts that escrow scholarship funds. Each of those contracts will pay the scholarship funds to a specific address if an oracle attests that they have earned the scholarship; otherwise the funds are returned to the multisig address. This contract is secure against misbehavior of the oracle in that the oracle cannot steal funds: the worst it can do is mis-award scholarships. The list of scholars' addresses must be defined before the contracts are created and, for security, the holders of the multisig keys must review the transaction they sign to ensure that it does indeed create escrowed scholarships for that list of addresses. Conceptually, each contract can be represented as the Blockly diagram below, where `Scholar` is the address of a particular scholar.

![Marlowe contract for a scholarship](contract-conceptual.png)

Thus, funds flow in the following manner.

![Flow of transactions](flow.png)

A [video demonstration of this example](https://youtu.be/H4PdCJvt7fQ) is available.

## Constants

It is convenient to define serveral constants.

In [1]:
ADA=1000000

In [2]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))

## Temporary folders

In [3]:
mkdir -p contracts states marlowes txs

## Network

Use the preproduction network.

In [4]:
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/preprod/node.socket
export CARDANO_TESTNET_MAGIC=1

## Parties to the contracts

Set the signing keys and addresses for parties to the contracts.

### Verifier

The *Verifier* authorizes payment of the scholarships.

In [5]:
VERIFIER_NAME=m.herbert
VERIFIER_SKEY=../keys/$VERIFIER_NAME.skey
VERIFIER_ADDR=$(cat ../keys/$VERIFIER_NAME.testnet.address)
echo "$VERIFIER_NAME @ $VERIFIER_ADDR"

m.herbert @ addr_test1vq08ewa9gw55z4wfd8fqexgkm8eyukm93qa2vr4w2u22j2g6ktqxw


Tidy the UTxOs at the verifier's address.

In [6]:
/extra/iohk/bin/marlowe-cli util clean \
  --change-address "$VERIFIER_ADDR" \
  --required-signer "$VERIFIER_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "79e8931f15eaf91ae625f8b08a95d003cf8e8fcbfdcc10d3583f664a06957b80"


View the funds at the verifier's address.

In [7]:
cardano-cli query utxo \
  --testnet-magic "$CARDANO_TESTNET_MAGIC" \
  --address "$VERIFIER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
79e8931f15eaf91ae625f8b08a95d003cf8e8fcbfdcc10d3583f664a06957b80     0        252182765 lovelace + TxOutDatumNone
79e8931f15eaf91ae625f8b08a95d003cf8e8fcbfdcc10d3583f664a06957b80     1        2000000 lovelace + 2 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.6d2e68657262657274 + TxOutDatumNone


### Scholars

Normally, the list of addresses for the *Scholars* would be provided. Here, for convenience, we justderive each scholar's address from the same root private key.

In [8]:
ROOT_PRV=../keys/william-shakespeare.root.prv

Also for convenience, the scholars will share the same stake key.

In [9]:
cardano-wallet key child 1852H/1815H/0H/2/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file ../keys/william-shakespeare.stake.vkey
STAKE_ADDR=$(cardano-cli stake-address build --testnet-magic $CARDANO_TESTNET_MAGIC --stake-verification-key-file ../keys/william-shakespeare.stake.vkey)
echo "Stake address = $STAKE_ADDR"

Stake address = stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


Set the number of scholars.

In [10]:
NUMBER_OF_SCHOLARS=100

Compute the addresses of the scholars.

In [11]:
for j in $(seq 1 $NUMBER_OF_SCHOLARS)
do
  SCHOLAR_SKEY[$j]=../keys/scholar-$j.skey
  SCHOLAR_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$j < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${SCHOLAR_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${SCHOLAR_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file ../keys/william-shakespeare.stake.vkey \
  )
  echo "Scholar $j = 1852H/1815H/0H/0/$((1 + j)) = ${SCHOLAR_ADDR[$j]}"
done

Scholar 1 = 1852H/1815H/0H/0/2 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
Scholar 2 = 1852H/1815H/0H/0/3 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp
Scholar 3 = 1852H/1815H/0H/0/4 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3
Scholar 4 = 1852H/1815H/0H/0/5 = addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr
Scholar 5 = 1852H/1815H/0H/0/6 = addr_test1qqxrzfv0v8l0lahyc5qsgpug66dafp8njl278g0pter7e3xr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqcjuuw0
Scholar 6 = 1852H/1815H/0H/0/7 = addr_test1qqqfzyuqkvrr7ajgk887sqac8lgzatc6el9uxke4x5g5dg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqkmrpwu
Scholar 7 = 1852H/1815H/0H/0/8 = addr_test1qp77epd4gy5hrt8pnh6h73se4durwrrkes3u0vj6tycwsq7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq7kyfpt
Schola

## Other addresses used in this example

### Administartor

The *Administrator* simply provides funds for the example and submits the creation transaction.

In [12]:
ADMINISTRATOR_NAME=j.lumley
ADMINISTRATOR_SKEY=../keys/$ADMINISTRATOR_NAME.skey
ADMINISTRATOR_ADDR=$(cat ../keys/$ADMINISTRATOR_NAME.testnet.address)
echo "$ADMINISTRATOR_NAME @ $ADMINISTRATOR_ADDR"

j.lumley @ addr_test1vrsuucqupq5xz7dw0tnw7w3cyck5zyk2kpnr56xlhr57m3gll8m84


Tidy the UTxOs at the administrator's address.

In [13]:
/extra/iohk/bin/marlowe-cli util clean \
  --change-address "$ADMINISTRATOR_ADDR" \
  --required-signer "$ADMINISTRATOR_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "d3af75f36ed8331ade46ec82ed22ad5b046e39dfe3365cd1a6f72f9e133f7dc3"


View the administrator's funds.

In [14]:
cardano-cli query utxo \
  --testnet-magic "$CARDANO_TESTNET_MAGIC" \
  --address "$ADMINISTRATOR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d3af75f36ed8331ade46ec82ed22ad5b046e39dfe3365cd1a6f72f9e133f7dc3     0        41688900411 lovelace + TxOutDatumNone
d3af75f36ed8331ade46ec82ed22ad5b046e39dfe3365cd1a6f72f9e133f7dc3     1        2000000 lovelace + 2 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.6a2e6c756d6c6579 + TxOutDatumNone


### Treasurers

The scholarship funds are held at a multisig address. In this example, we have three *Treasurers* that collectively contract that multisig.

In [15]:
TREASURER_1_NAME=c.marlowe
TREASURER_1_SKEY=../keys/$TREASURER_1_NAME.skey
TREASURER_1_ADDR=$(cat ../keys/$TREASURER_1_NAME.testnet.address)
TREASURER_1_PKH=$(cardano-cli key verification-key --signing-key-file $TREASURER_1_SKEY --verification-key-file /dev/stdout | cardano-cli address key-hash --verification-key-file /dev/stdin)
echo "$TREASURER_1_NAME @ $TREASURER_1_ADDR"

c.marlowe @ addr_test1vrtntkszteptml4e9ce9l3fsmgavwv4ywunvdnhxv6nw5ksq6737a


In [16]:
TREASURER_2_NAME=e.cary
TREASURER_2_SKEY=../keys/$TREASURER_2_NAME.skey
TREASURER_2_ADDR=$(cat ../keys/$TREASURER_2_NAME.testnet.address)
TREASURER_2_PKH=$(cardano-cli key verification-key --signing-key-file $TREASURER_2_SKEY --verification-key-file /dev/stdout | cardano-cli address key-hash --verification-key-file /dev/stdin)
echo "$TREASURER_2_NAME @ $TREASURER_2_ADDR"

e.cary @ addr_test1vpkedn0l6slaj8fzkx08qyce9vpreuu3spmnlxkepx4757grxzyzf


In [17]:
TREASURER_3_NAME=j.webster
TREASURER_3_SKEY=../keys/$TREASURER_3_NAME.skey
TREASURER_3_ADDR=$(cat ../keys/$TREASURER_3_NAME.testnet.address)
TREASURER_3_PKH=$(cardano-cli key verification-key --signing-key-file $TREASURER_3_SKEY --verification-key-file /dev/stdout | cardano-cli address key-hash --verification-key-file /dev/stdin)
echo "$TREASURER_3_NAME @ $TREASURER_3_ADDR"

j.webster @ addr_test1vzgrqnlp6elmettvuelx5vkn0uxhtu2ewqdhx297ukgjmjgpss5k0


## Contract Parameters

### Deadlines

Start the contract at the current time.

In [18]:
NOW=$(($(date -u +%s) * 1000))
echo "NOW = $NOW = $(date -d @$((NOW / 1000)))"

NOW = 1683390067000 = Sat May  6 10:21:07 AM MDT 2023


In [19]:
REDEMPTION_DEADLINE=$(($(date -u +%s) * SECOND + 2 * HOUR))
echo "REDEMPTION_DEADLINE = $REDEMPTION_DEADLINE = $(date -d @$((REDEMPTION_DEADLINE / 1000)))"

REDEMPTION_DEADLINE = 1683397269000 = Sat May  6 12:21:09 PM MDT 2023


### Scholarships

In [20]:
NUMBER_OF_SCHOLARSHIPS=$NUMBER_OF_SCHOLARS
echo "NUMBER_OF_SCHOLARSHIPS = $NUMBER_OF_SCHOLARSHIPS"

NUMBER_OF_SCHOLARSHIPS = 100


In [21]:
SCHOLARSHIP_AMOUNT=$((400 * ADA))
echo "SCHOLARSHIP_AMOUNT = $SCHOLARSHIP_AMOUNT lovelace"

SCHOLARSHIP_AMOUNT = 400000000 lovelace


## Treasury

The *Treasury* is a multisig address that holds the scholarship funds.

### Multisig Script

In this example we use an Allegra two-of-three multisig script.

In [22]:
yaml2json << EOI > multisig.json
type: atLeast
required: 2
scripts:
- keyHash: $TREASURER_1_PKH
  type: sig
- keyHash: $TREASURER_2_PKH
  type: sig
- keyHash: $TREASURER_3_PKH
  type: sig
EOI
cat multisig.json

{"required":2,"scripts":[{"keyHash":"d735da025e42bdfeb92e325fc530da3ac732a47726c6cee666a6ea5a","type":"sig"},{"keyHash":"6d96cdffd43fd91d22b19e7013192b023cf39180773f9ad909abea79","type":"sig"},{"keyHash":"90304fe1d67fbcad6ce67e6a32d37f0d75f159701b7328bee5912dc9","type":"sig"}],"type":"atLeast"}


Compute the multisig address.

In [23]:
TREASURY_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-script-file multisig.json --stake-verification-key-file ../keys/william-shakespeare.stake.vkey)
echo "TREASURY_ADDR = $TREASURY_ADDR"

TREASURY_ADDR = addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0


### Fund the treasury

In this example, the administrator will fund the treasury. Normally, that would be funded by contributors to the scholarship fund.

First find a UTxO that is large enough to fund the treasury.

In [24]:
TX_IN=$(
marlowe-cli util select \
  --lovelace-only "$((NUMBER_OF_SCHOLARSHIPS * SCHOLARSHIP_AMOUNT + 30 * ADA))" \
  "$ADMINISTRATOR_ADDR" \
| sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;2,$d' \
)
echo "TX_IN = $TX_IN"

TX_IN = d3af75f36ed8331ade46ec82ed22ad5b046e39dfe3365cd1a6f72f9e133f7dc3#0


Now fund the treasury.

In [25]:
TX[1]=$(
/extra/iohk/bin/marlowe-cli transaction simple \
  --tx-in "$TX_IN" \
  --tx-out "$TREASURY_ADDR+$((NUMBER_OF_SCHOLARSHIPS * SCHOLARSHIP_AMOUNT))" \
  --change-address "$ADMINISTRATOR_ADDR" \
  --required-signer "$ADMINISTRATOR_SKEY" \
  --out-file /dev/null \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[1] = ${TX[1]}"

TX[1] = 0ebeb800c9d95f63d54711cfd111b3910707eb33bd1716c8f7591f24423024fd


View the funds in the treasury.

In [26]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$TREASURY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0ebeb800c9d95f63d54711cfd111b3910707eb33bd1716c8f7591f24423024fd     1        40000000000 lovelace + TxOutDatumNone


## Contracts

Create a separate Marlowe contract for each scholar.

Although each Marlowe contract conceptually includes a `Deposit` block, in practice we bypass that block by initializing the contract's state with the scholarship funds already deposited. This eliminates some fees and it permits the multisig key holders to only sign the creation transaction, not each of the 100 deposit transactions. The actual contract on the chain is as shown below.

![Scholarship contract](contract-actual.png)

For the first two contracts, we'll set the redemption deadline to the past so that we can execute the timeout case where funds are returned to the treasury. (This is simply for convenience of this example, so we don't have to wait a long time for the deadline to pass.)

In [27]:
for i in $(seq 1 2)
do
yaml2json << EOI > "contracts/$i.json"
when:
- case:
    for_choice:
      choice_name: Pay Scholarship
      choice_owner:
        address: $VERIFIER_ADDR
    choose_between:
    - from: 0
      to: 0
  then:
    from_account:
      address: $TREASURY_ADDR
    token:
      currency_symbol: ''
      token_name: ''
    pay: $SCHOLARSHIP_AMOUNT
    to:
      party:
        address: ${SCHOLAR_ADDR[$i]}
    then: close
timeout: $NOW
timeout_continuation: close
EOI
done

cat contracts/1.json

{"timeout":1683390067000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Pay Scholarship","choice_owner":{"address":"addr_test1vq08ewa9gw55z4wfd8fqexgkm8eyukm93qa2vr4w2u22j2g6ktqxw"}}},"then":{"from_account":{"address":"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0"},"pay":400000000,"then":"close","to":{"party":{"address":"addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p"}},"token":{"currency_symbol":"","token_name":""}}}]}


Create the remaining contracts normally, with their redemption deadline in the future.

In [28]:
for i in $(seq 3 $NUMBER_OF_SCHOLARSHIPS)
do
yaml2json << EOI > "contracts/$i.json"
when:
- case:
    for_choice:
      choice_name: Pay Scholarship
      choice_owner:
        address: $VERIFIER_ADDR
    choose_between:
    - from: 0
      to: 0
  then:
    from_account:
      address: $TREASURY_ADDR
    token:
      currency_symbol: ''
      token_name: ''
    pay: $SCHOLARSHIP_AMOUNT
    to:
      party:
        address: ${SCHOLAR_ADDR[$i]}
    then: close
timeout: $REDEMPTION_DEADLINE
timeout_continuation: close
EOI
done

cat contracts/3.json

{"timeout":1683397269000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Pay Scholarship","choice_owner":{"address":"addr_test1vq08ewa9gw55z4wfd8fqexgkm8eyukm93qa2vr4w2u22j2g6ktqxw"}}},"then":{"from_account":{"address":"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0"},"pay":400000000,"then":"close","to":{"party":{"address":"addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3"}},"token":{"currency_symbol":"","token_name":""}}}]}


Make the initial state of each contract contain the scholarship funds.

In [29]:
for i in $(seq 1 $NUMBER_OF_SCHOLARSHIPS)
do
yaml2json << EOI > "states/$i.json"
accounts:
- - - address: $TREASURY_ADDR
    - currency_symbol: ''
      token_name: ''
  - $SCHOLARSHIP_AMOUNT
boundValues: []
choices: []
minTime: 1
EOI
done

cat states/1.json

{"accounts":[[[{"address":"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0"},{"currency_symbol":"","token_name":""}],400000000]],"boundValues":[],"choices":[],"minTime":1}


Compute the datum hash for each contract.

In [30]:
for i in $(seq 1 $NUMBER_OF_SCHOLARSHIPS)
do
marlowe-cli run initialize \
  --permanently-without-staking \
  --stake-address "$STAKE_ADDR" \
  --contract-file "contracts/$i.json" \
  --state-file "states/$i.json" \
  --out-file "marlowes/${i}a.json"
DATUM_HASH[$i]=$(
marlowe-cli contract datum \
 --contract-file "contracts/$i.json" \
 --state-file "states/$i.json" \
 --out-file /dev/null
)
echo "DATUM_HASH[$i] = ${DATUM_HASH[$i]}"
done


DATUM_HASH[1] = 54a6d8e970e757f815dea5fbb21b65aee7f7a1b5e4852ff83dff684f1cb2abdc
DATUM_HASH[2] = 318bd42feb3a44c971fe13048d7b5b491124e96413579be3e1a8032c92ca1b19
DATUM_HASH[3] = f889a9a9d3f2fe7e593738de4b2d9c6a52a80cdf13ab41264c7306771774c306
DATUM_HASH[4] = 4d7ed021621bed1c790791c99cbaa0c80d0a15f2502eb5257c7f75d3240d686a
DATUM_HASH[5] = 54c6924a6e7bc67be4c968de22212d8a99ad933c227f9df3870ee34fe74eaf30
DATUM_HASH[6] = 90bdd0273982bacb3182ccf9484a7dfac99de857967fde2cc72d8721d13ffdd2
DATUM_HASH[7] = a633fa3d2772c3c981ded298b91fcb5868e693d64a81a42bccaae0272251c0fe
DATUM_HASH[8] = 7ff00e4c787ae4fb29a8fd0c05c63e293908974893921915974fb8d975df7b7b
DATUM_HASH[9] = 1e59cf0685d8615a74709e75e88078ed5624b842e709c45a8c2ba77d8d660ac2
DATUM_HASH[10] = 99cd51948fe6b6e762dea2d02c24f58611e5a768dc1fe15f93bdc0233bf428e1
DATUM_HASH[11] = a1448ee00466479ab1736bed8157737040b949cecc8c3d2b6d335aa350ba1652
DATUM_HASH[12] = 0da707efde65a6700d00a44f5e7640bb84f2d1a254184f01e4d22f891e26b73f
DATUM_HASH[13] = c1d6cc2b

## Creation transaction

The creation transaction will move funds from the treasury into separate contracts for each scholar. These contracts all reside at the address of Marlowe's semantics validator.

In [31]:
MARLOWE_ADDR=$(jq -r '.tx.marloweValidator.address' marlowes/1a.json)
echo "MARLOWE_ADDR = $MARLOWE_ADDR"

MARLOWE_ADDR = addr_test1zqhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535mkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqc6s825


The administrator will pay the transaction fee, so find one of their UTxOs with sufficient funds.

In [32]:
TX_IN=$(
marlowe-cli util select \
  --lovelace-only "$((10 * ADA))" \
  "$ADMINISTRATOR_ADDR" \
| sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;2,$d' \
)
echo "TX_IN = ${TX_IN}"

TX_IN = 0ebeb800c9d95f63d54711cfd111b3910707eb33bd1716c8f7591f24423024fd#0


Now build a transaction that sends the treasury funds to the scholarship contract.

***Important notes:***
* If the treasury contains excess funds, a change output back to the treasury must be added to the command below.
* This command assumes that treasury funds have been consolidated into a single UTxO.

In [33]:
cardano-cli transaction build \
  --testnet-magic "$CARDANO_TESTNET_MAGIC" \
  --babbage-era \
  --tx-in "$TX_IN" \
  --tx-in "${TX[1]}#1" \
    --tx-in-script-file multisig.json \
  $(for i in $(seq 1 $NUMBER_OF_SCHOLARSHIPS); do echo "--tx-out $MARLOWE_ADDR+$SCHOLARSHIP_AMOUNT --tx-out-datum-hash ${DATUM_HASH[$i]}"; done) \
  --change-address "$ADMINISTRATOR_ADDR" \
  --witness-override 4 \
  --out-file txs/2.unsigned

Estimated transaction fee: Lovelace 641053


Submit the transaction to create the contracts.

In [34]:
TX[2]=$(
/extra/iohk/bin/marlowe-cli transaction submit \
  --tx-body-file txs/2.unsigned \
  --required-signer "$ADMINISTRATOR_SKEY" \
  --required-signer "$TREASURER_1_SKEY" \
  --required-signer "$TREASURER_2_SKEY" \
  --required-signer "$TREASURER_3_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[2] = ${TX[2]}"  

TX[2] = 48f23b4f32006849a0f5ed76f32229d0f615b0bb2b79a11f329864c8b0c237ae


View the contracts at the address of the Marlowe semantics validator.

In [35]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
48f23b4f32006849a0f5ed76f32229d0f615b0bb2b79a11f329864c8b0c237ae     1        400000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "54a6d8e970e757f815dea5fbb21b65aee7f7a1b5e4852ff83dff684f1cb2abdc"
48f23b4f32006849a0f5ed76f32229d0f615b0bb2b79a11f329864c8b0c237ae     2        400000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "318bd42feb3a44c971fe13048d7b5b491124e96413579be3e1a8032c92ca1b19"
48f23b4f32006849a0f5ed76f32229d0f615b0bb2b79a11f329864c8b0c237ae     3        400000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "f889a9a9d3f2fe7e593738de4b2d9c6a52a80cdf13ab41264c7306771774c306"
48f23b4f32006849a0f5ed76f32229d0f615b0bb2b79a11f329864c8b0c237ae     4        400000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "4d7ed021621bed1c790791c99cbaa0c80d0a15f2502eb5257c7f75d3240d686a"
48f23b4f3200

## Distribute the scholarships

For each scholar, the verifier attests whether they have earned the scholarship. This causes the scholarship funds to be sent to that scholar's address. If the verifier does not make the attestation, then the funds are returned to the treasury.

First execute the first two scholarships, where the verifier ***does not*** make the attestation, so the funds are returned to the treasury.

In [37]:
for i in $(seq 1 2)
do

n=$((i + 2))

echo
echo "----- Contract $i -----"
echo

marlowe-cli run prepare \
  --invalid-before "$((($(date -u +%s) - 1 * 60) * 1000))" \
  --invalid-hereafter "$((($(date -u +%s) + 5 * 60) * 1000))" \
  --marlowe-file "marlowes/${i}a.json" \
  --out-file "marlowes/${i}b.json"

TX[$n]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --tx-in-marlowe "${TX[2]}#$i" \
  --marlowe-in-file "marlowes/${i}a.json" \
  --marlowe-out-file "marlowes/${i}b.json" \
  --change-address "$VERIFIER_ADDR" \
  --required-signer "$VERIFIER_SKEY" \
  --out-file /dev/null \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[$n] = ${TX[$n]}"  

echo
echo "Funds at scholar $i's address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${SCHOLAR_ADDR[$i]}"

echo
echo "Funds in the treasury:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$TREASURY_ADDR"

done


----- Contract 1 -----

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1683390534000},POSIXTime {getPOSIXTime = 1683390894999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683390534000},POSIXTime {getPOSIXTime = 1683390894999}), txInputs = []}
Payment 1
  Acccount: "\"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0\""
  Payee: Party "\"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0\""
  Ada: Lovelace {getLovelace = 400000000}
TX[3] = e09c80cfd8ae1fd99711e34f63e637255289430ff5b17a4bdd7d901069a4f614

Funds at scholar 1's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------

Funds in the treasury:
                           TxHash                                 TxIx        Amount
-------------

Now execute the remain scholarships, where the verifier ***does*** make the attestation.

In [38]:
for i in $(seq 3 $NUMBER_OF_SCHOLARSHIPS)
do

n=$((i + 2))

echo
echo "----- Contract $i -----"
echo

marlowe-cli run prepare \
  --choice-party "$VERIFIER_ADDR" \
  --choice-name "Pay Scholarship" \
  --choice-number 0 \
  --invalid-before "$((($(date -u +%s) - 1 * 60) * 1000))" \
  --invalid-hereafter "$((($(date -u +%s) + 5 * 60) * 1000))" \
  --marlowe-file "marlowes/${i}a.json" \
  --out-file "marlowes/${i}b.json"

TX[$n]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --tx-in-marlowe "${TX[2]}#$i" \
  --marlowe-in-file "marlowes/${i}a.json" \
  --marlowe-out-file "marlowes/${i}b.json" \
  --change-address "$VERIFIER_ADDR" \
  --required-signer "$VERIFIER_SKEY" \
  --out-file /dev/null \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[$n] = ${TX[$n]}"  

echo
echo "Funds at scholar $i's address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${SCHOLAR_ADDR[$i]}"

done


----- Contract 3 -----

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1683390628000},POSIXTime {getPOSIXTime = 1683390988999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683390628000},POSIXTime {getPOSIXTime = 1683390988999}), txInputs = [NormalInput (IChoice (ChoiceId "Pay Scholarship" "\"addr_test1vq08ewa9gw55z4wfd8fqexgkm8eyukm93qa2vr4w2u22j2g6ktqxw\"") 0)]}
Payment 1
  Acccount: "\"addr_test1zq6zvzptxvhdhd2392qe30ar9htwgczyg5802he0luk2w3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqlk60k0\""
  Payee: Party "\"addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3\""
  Ada: Lovelace {getLovelace = 400000000}
TX[5] = 27eaf5444c01291bd205a1cce1ec70202ed89fd4fe7842f0da94ef2053769c48

Funds at scholar 3's address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


## Cleanup

Return the funds to the administrator.

In [39]:
cardano-cli transaction build \
  --testnet-magic "$CARDANO_TESTNET_MAGIC" \
  --babbage-era \
  $(for i in $(seq 1 2); do echo "--tx-in ${TX[$((i+2))]}#1 --tx-in-script-file multisig.json"; done) \
  $(for i in $(seq 3 $NUMBER_OF_SCHOLARSHIPS); do echo "--tx-in ${TX[$((i+2))]}#1"; done) \
  --change-address "$ADMINISTRATOR_ADDR" \
  --witness-override $((2 + NUMBER_OF_SCHOLARSHIPS)) \
  --out-file txs/cleanup.unsigned

Estimated transaction fee: Lovelace 774241


In [40]:
/extra/iohk/bin/marlowe-cli transaction submit \
  --tx-body-file txs/cleanup.unsigned \
  --required-signer "$ADMINISTRATOR_SKEY" \
  --required-signer "$TREASURER_1_SKEY" \
  --required-signer "$TREASURER_2_SKEY" \
  --required-signer "$TREASURER_3_SKEY" \
  $(for i in $(seq 3 $NUMBER_OF_SCHOLARSHIPS); do echo "--required-signer ${SCHOLAR_SKEY[$i]}"; done) \
  --timeout 600

TxId "63e5ebbc58b84bc441d67dbc484c8f5425b5a1b25011ae04ed3ba76f9ad40e00"


In [41]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$TREASURY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [42]:
cardano-cli query utxo \
  --testnet-magic "$CARDANO_TESTNET_MAGIC" \
  $(for i in $(seq 1 $NUMBER_OF_SCHOLARSHIPS); do echo "--address ${SCHOLAR_ADDR[$i]}"; done)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
